In [1]:
import pandas as pd
%matplotlib inline
%run Pipeline//upload_and_vizualize 
%run Pipeline//classify_and_evaluate 
%run Pipeline//aux
%run Pipeline//ULAB_ML_Pipeline
%run processing
import numpy as np 
import matplotlib.pyplot as plt
import pylab
import sys
import random
import sklearn as sk 
import json 
import re
from multiprocessing import Pool
from functools import partial
from time import time
from sklearn import svm, ensemble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split, KFold
from sklearn.preprocessing import *
from sklearn.feature_selection import RFE
from sklearn.grid_search import ParameterGrid
from sklearn.svm import LinearSVC
from sklearn.metrics import *
import csv
from errno import EEXIST
from os import makedirs,path


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
FACILITY = 'ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv'
VIOLATION = 'ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv'
INSPECTION = 'ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv'
FORMALACT = 'ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv'
INFORMALACT = 'ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv'
STACKTEST = 'ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv'
TITLEV = 'ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv'

#facilities = read_file('ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv')
#pd_violations = read_file('ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
#pd_inspection = read_file('ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
#pd_formalact = read_file('ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv')
#pd_informalact = read_file('ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv')
#pd_stacktest = read_file('ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv')
#pd_titlev = read_file('ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv')

In [3]:
## FROM PREPROCESSING ##

#interest_var = ['PGM_SYS_ID','ACTIVITY_ID','AGENCY_TYPE_DESC','STATE_CODE','AIR_LCON_CODE','COMP_DETERMINATION_UID','ENF_RESPONSE_POLICY_CODE','PROGRAM_CODES']
def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)

def convert_to_year(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.year)

def to_date_time(df, date_format, date_col):
    #add datetime column
    df[date_col] = df[date_col].apply(convert_to_datetime, date_format=date_format)
    df[date_col+'_year'] = df[date_col].apply(convert_to_year)

    return df

def filter_date(df, date_format, date_col, start=None, end=None):
    df = to_date_time(df, date_format, date_col)
    
    
    if start:
        timestart = dt.strptime(start,"%Y/%m/%d")
        #print(start)
        df = df[df[date_col] >= timestart ]
        #print(df.head())
    if end:
        timeend = dt.strptime(end,"%Y/%m/%d")
        #print(end)
        df = df[df[date_col] <= timeend ]
        #print(df.head())
    
    return df

def filter_col(df, fac_id, features, date_col):
    #filter needed
    df = df[[fac_id] + [date_col] + [date_col+'_year'] + features]
    return df

In [4]:
## CONFIG DATA ##
START_DATE= '2007/01/01'
END_DATE = '2016/12/31'
fac_id = 'PGM_SYS_ID'
df_dict ={'violation': {'interest_var': ['AGENCY_TYPE_DESC','STATE_CODE','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_RESOLVED_DATE'],
                         'date_col': 'HPV_DAYZERO_DATE',
                       'date_format':'%m-%d-%Y'},
          
           'inspection': {'interest_var': ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES'],
                         'date_col': 'ACTUAL_END_DATE',
                         'date_format':'%m-%d-%Y'},
          
           'stacktest': {'interest_var':['COMP_MONITOR_TYPE_CODE','POLLUTANT_CODES','AIR_STACK_TEST_STATUS_CODE'],
                        'date_col': 'ACTUAL_END_DATE',
                        'date_format':'%m/%d/%Y'},
          
           'titlev':{'interest_var':['COMP_MONITOR_TYPE_CODE','FACILITY_RPT_DEVIATION_FLAG'],
                        'date_col': 'ACTUAL_END_DATE',
                    'date_format':'%m/%d/%Y'},
          
           'formalact':{'interest_var':['ENF_TYPE_CODE','PENALTY_AMOUNT'],
                        'date_col': 'SETTLEMENT_ENTERED_DATE',
                       'date_format':'%m/%d/%Y'},
          
           'informalact':{'interest_var':['ENF_TYPE_CODE'],
                        'date_col': 'ACHIEVED_DATE',
                         'date_format':'%m/%d/%Y'}}

#general_df = pd.DataFrame()

In [5]:
for i, v in df_dict.items():
    print(v['date_col'])

HPV_DAYZERO_DATE
ACTUAL_END_DATE
ACTUAL_END_DATE
ACTUAL_END_DATE
SETTLEMENT_ENTERED_DATE
ACHIEVED_DATE


In [6]:
def general_read_file(df_dict, start_date, end_date):
    #df = pd.DataFrame()
    
    violation = []
    inspection = []
    stacktest = []
    titlev = []
    formalact = []
    informalact = []
    
    for table, var in df_dict.items():
        date_col = var['date_col']
        features = var['interest_var']
        DATE_FORMAT = var['date_format']
        
        if table == 'violation':
            violation = read_file(VIOLATION)
            violation = violation[violation['ENF_RESPONSE_POLICY_CODE'] != 'FRV']
            
            violation = filter_date(violation, DATE_FORMAT, date_col, start=start_date, end=end_date)
            violation = filter_col(violation, fac_id, features, date_col)
        
        elif table == 'inspection':
            inspection = read_file(INSPECTION)
            
            inspection = filter_date(inspection, DATE_FORMAT, date_col, start=start_date, end=end_date)
            inspection = filter_col(inspection, fac_id, features, date_col)
        
        elif table == 'titlev':
            titlev = read_file(TITLEV)
            
            titlev = filter_date(titlev, DATE_FORMAT, date_col, start=start_date, end=end_date)
            titlev = filter_col(titlev, fac_id, features, date_col)
        
        elif table == 'stacktest':
            stacktest = read_file(STACKTEST)
            
            stacktest = filter_date(stacktest, DATE_FORMAT, date_col, start=start_date, end=end_date)
            stacktest = filter_col(stacktest, fac_id, features, date_col)
        
        elif table == 'formalact':
            formalact = read_file(FORMALACT)
            
            formalact = filter_date(formalact, DATE_FORMAT, date_col, start=start_date, end=end_date)
            formalact = filter_col(formalact, fac_id, features, date_col)
        
        elif table == 'informalact':
            informalact = read_file(INFORMALACT)
            
            informalact = filter_date(informalact, DATE_FORMAT, date_col, start=start_date, end=end_date)
            informalact = filter_col(informalact, fac_id, features, date_col)
        
    return (violation, inspection, titlev, stacktest, formalact, informalact)

In [18]:
## READ INITIAL FILTERED FILE ##
violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)

In [42]:
def add_dummy(df, variable_list, sep_char = None, drop_one=False, drop_original=False):
    '''
    Input: 
        - df: pandas dataframe
        - variable_list: a list of variables to dummitize
        - drop_one: whether to drop first dummy
        - drop_original: whether to drop original categorical variable
    Output: dataframe with tht dummy variables added
    '''
    for variable in variable_list:
        if sep_char:
            df_dummy = df[variable].str.get_dummies(sep=sep_char)
            df_dummy.columns = [variable+ '_' +str(col) for col in df_dummy.columns]

        else:
            df_dummy = pd.get_dummies(df[variable], drop_first=drop_one, prefix = variable)
        
        df = pd.concat([df, df_dummy], axis=1)
        if drop_original:
            df = df.drop(variable, 1)
    return df

In [43]:
viol = violation
viol

,PGM_SYS_ID,HPV_DAYZERO_DATE,HPV_DAYZERO_DATE_year,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,ENF_RESPONSE_POLICY_CODE,POLLUTANT_CODES,PROGRAM_CODES,HPV_RESOLVED_DATE
5,CT0000000900508907,2010-03-17,2010,State,CT,None,HPV,300000329,CAAFESOP,11-02-2010
6,CT0000000900100078,2007-08-08,2007,State,CT,None,HPV,300000329,CAAFESOP,09-24-2009
7,CT0000000901501125,2013-04-26,2013,State,CT,None,HPV,10193,CAATVP,06-25-2014
9,CT0000000900109058,2008-01-17,2008,State,CT,None,HPV,300000005,CAAFESOP,01-06-2009
13,CT0000000901100054,2008-04-08,2008,State,CT,None,HPV,10461 300000005,CAATVP,12-10-2008
16,CT0000000901100038,2009-10-01,2009,State,CT,None,HPV,300000005,CAASIP,03-25-2011
18,CT0000000900900016,2010-04-26,2010,State,CT,None,HPV,300000329,CAANSPS CAANSR CAASIP CAATVP,10-15-2012
24,MA0000002512000866,2010-03-01,2010,State,MA,None,HPV,10461,CAASIP CAATVP,12-31-2011
29,CT0000000900909910,2010-07-27,2010,State,CT,None,HPV,300000298,CAANSR,08-05-2011
30,CT0000000900909936,2010-05-18,2010,State,CT,None,HPV,300000005,CAASIP CAATVP,02-01-2011


In [10]:
violation.isnull().sum()

PGM_SYS_ID                     0
HPV_DAYZERO_DATE               0
HPV_DAYZERO_DATE_year          0
AGENCY_TYPE_DESC               0
STATE_CODE                   741
AIR_LCON_CODE               7620
ENF_RESPONSE_POLICY_CODE       0
POLLUTANT_CODES              290
PROGRAM_CODES                  0
HPV_RESOLVED_DATE           2229
dtype: int64

In [50]:
violation['POLLUTANT_CODES'].str.get_dummies(sep=' ').columns

Index(['10193', '10214', '10243', '10305', '10358', '10361', '10366', '10373',
       '10461', '10486',
       ...
       '300000415', '300000452', '5409', '5474', '5617', '603', '6562', '6696',
       '7003', '8240'],
      dtype='object', length=137)

In [51]:
violation['AGENCY_TYPE_DESC'].value_counts()

State       6879
Local       4658
U.S. EPA     741
Name: AGENCY_TYPE_DESC, dtype: int64

In [44]:
## Replace NaN with 'None' (string)
viol_nan = viol.columns[viol.isnull().any()].tolist()
values = ['None']*len(viol_nan)
replace_with_value(viol, viol_nan, values)

In [47]:
viol.isnull().sum()

PGM_SYS_ID                   0
HPV_DAYZERO_DATE             0
HPV_DAYZERO_DATE_year        0
AGENCY_TYPE_DESC             0
STATE_CODE                   0
AIR_LCON_CODE                0
ENF_RESPONSE_POLICY_CODE     0
POLLUTANT_CODES              0
PROGRAM_CODES                0
HPV_RESOLVED_DATE            0
AGENCY_TYPE_DESC_Local       0
AGENCY_TYPE_DESC_State       0
AGENCY_TYPE_DESC_U.S. EPA    0
PROGRAM_CODES_CAAAR          0
PROGRAM_CODES_CAACFC         0
PROGRAM_CODES_CAAEP          0
PROGRAM_CODES_CAAFESOP       0
PROGRAM_CODES_CAAFIP         0
PROGRAM_CODES_CAAGACTM       0
PROGRAM_CODES_CAAGHG         0
PROGRAM_CODES_CAAMACT        0
PROGRAM_CODES_CAANAM         0
PROGRAM_CODES_CAANESH        0
PROGRAM_CODES_CAANFRP        0
PROGRAM_CODES_CAANSPS        0
PROGRAM_CODES_CAANSPSM       0
PROGRAM_CODES_CAANSR         0
PROGRAM_CODES_CAAOP          0
PROGRAM_CODES_CAAOTH         0
PROGRAM_CODES_CAAPARGDC      0
PROGRAM_CODES_CAAPSD         0
PROGRAM_CODES_CAAPSDPR       0
PROGRAM_

In [45]:
## Dummitize 
viol = add_dummy(viol, ['AGENCY_TYPE_DESC'])
viol = add_dummy(viol, ['PROGRAM_CODES'], sep_char = ' ')

In [46]:
viol

,PGM_SYS_ID,HPV_DAYZERO_DATE,HPV_DAYZERO_DATE_year,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,ENF_RESPONSE_POLICY_CODE,POLLUTANT_CODES,PROGRAM_CODES,HPV_RESOLVED_DATE,...,PROGRAM_CODES_CAANSPSM,PROGRAM_CODES_CAANSR,PROGRAM_CODES_CAAOP,PROGRAM_CODES_CAAOTH,PROGRAM_CODES_CAAPARGDC,PROGRAM_CODES_CAAPSD,PROGRAM_CODES_CAAPSDPR,PROGRAM_CODES_CAASIP,PROGRAM_CODES_CAASPP,PROGRAM_CODES_CAATVP
5,CT0000000900508907,2010-03-17,2010,State,CT,None,HPV,300000329,CAAFESOP,11-02-2010,...,0,0,0,0,0,0,0,0,0,0
6,CT0000000900100078,2007-08-08,2007,State,CT,None,HPV,300000329,CAAFESOP,09-24-2009,...,0,0,0,0,0,0,0,0,0,0
7,CT0000000901501125,2013-04-26,2013,State,CT,None,HPV,10193,CAATVP,06-25-2014,...,0,0,0,0,0,0,0,0,0,1
9,CT0000000900109058,2008-01-17,2008,State,CT,None,HPV,300000005,CAAFESOP,01-06-2009,...,0,0,0,0,0,0,0,0,0,0
13,CT0000000901100054,2008-04-08,2008,State,CT,None,HPV,10461 300000005,CAATVP,12-10-2008,...,0,0,0,0,0,0,0,0,0,1
16,CT0000000901100038,2009-10-01,2009,State,CT,None,HPV,300000005,CAASIP,03-25-2011,...,0,0,0,0,0,0,0,1,0,0
18,CT0000000900900016,2010-04-26,2010,State,CT,None,HPV,300000329,CAANSPS CAANSR CAASIP CAATVP,10-15-2012,...,0,1,0,0,0,0,0,1,0,1
24,MA0000002512000866,2010-03-01,2010,State,MA,None,HPV,10461,CAASIP CAATVP,12-31-2011,...,0,0,0,0,0,0,0,1,0,1
29,CT0000000900909910,2010-07-27,2010,State,CT,None,HPV,300000298,CAANSR,08-05-2011,...,0,1,0,0,0,0,0,0,0,0
30,CT0000000900909936,2010-05-18,2010,State,CT,None,HPV,300000005,CAASIP CAATVP,02-01-2011,...,0,0,0,0,0,0,0,1,0,1


In [63]:
vtest = viol[['PGM_SYS_ID','AGENCY_TYPE_DESC','HPV_DAYZERO_DATE_year']]
pd.crosstab(vtest['PGM_SYS_ID'], [vtest['AGENCY_TYPE_DESC'],vtest['HPV_DAYZERO_DATE_year']]).

AGENCY_TYPE_DESC      Local                                              ...   \
HPV_DAYZERO_DATE_year  2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 ...    
PGM_SYS_ID                                                               ...    
0500000026055R0002        0    0    0    0    0    0    0    0    0    0 ...    
0500000026061R0001        0    0    0    0    0    0    0    0    0    0 ...    
0500000039067R0001        0    0    0    0    0    0    0    0    0    0 ...    
0500000055079R0018        0    0    0    0    0    0    0    0    0    0 ...    
0500018017R5001           0    0    0    0    0    0    0    0    0    0 ...    
0500026025R5001           0    0    0    0    0    0    0    0    0    0 ...    
0500026049R5001           0    0    0    0    0    0    0    0    0    0 ...    
0500026091R5001           0    0    0    0    0    0    0    0    0    0 ...    
0500026099R5001           0    0    0    0    0    0    0    0    0    0 ...    
0500039013R5003           0    0    0    0    0    0    0    0    0    0 ...    
080000004904700105        0    0    0    0    0    0    0    0    0    0 ...    
080000004904700106        0    0    0    0    0    0    0    0    0    0 ...    
080000004904700107        0    0    0    0    0    0    0    0    0    0 ...    
080000004904700108        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9300        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9301        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9356        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9358        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9360        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9401        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9402        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9404        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9405        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9444        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9550        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9551        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9605        0    0    0    0    0    0    0    0    0    0 ...    
0900000004013R9999        0    0    0    0    0    0    0    0    0    0 ...    
0900000004021R9001        0    0    0    0    0    0    0    0    0    0 ...    
0900000006001R9400        0    0    0    0    0    0    0    0    0    0 ...    
...                     ...  ...  ...  ...  ...  ...  ...  ...  ...  ... ...    
WY0000005600700048        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005600700069        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005601300027        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005601300028        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005602100001        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005602500005        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005602500006        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005602900012        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005602900014        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005602900015        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005603500019        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005603500020        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005603700001        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005603700002        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005603700005        0    0    0    0    0    0    0    0    0    0 ...    
WY0000005603700010        0    

In [40]:
inspection.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,STATE_EPA_FLAG,COMP_MONITOR_TYPE_CODE,PROGRAM_CODES
0,MI00000000000A1932,2015-06-09 00:00:00,2015,E,PCE,"CAAFESOP, CAASIP"
1,CASJV00006029S2076,2015-06-10 00:00:00,2015,L,FOO,CAATVP
2,CASJV00006039C2587,2015-06-08 00:00:00,2015,L,FOO,CAASIP
3,CASJV00006029S1234,2015-06-10 00:00:00,2015,L,FOO,CAATVP
4,CASJV00006099N7478,2015-06-11 00:00:00,2015,L,FOO,CAATVP


In [27]:
titlev.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,COMP_MONITOR_TYPE_CODE,FACILITY_RPT_DEVIATION_FLAG
0,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
1,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
2,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
3,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
4,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y


In [28]:
stacktest.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,COMP_MONITOR_TYPE_CODE,POLLUTANT_CODES,AIR_STACK_TEST_STATUS_CODE
0,RI0000004400900016,2007-10-03 00:00:00,2007,CST,Volatile organic compounds,NaN
1,CT0000000900900001,2007-01-17 00:00:00,2007,CST,Volatile organic compounds,NaN
2,CT0000000900900001,2007-01-18 00:00:00,2007,CST,Volatile organic compounds,NaN
3,ME0000002301300004,2007-05-30 00:00:00,2007,CST,Volatile organic compounds,NaN
4,MA0000002511900605,2011-09-28 00:00:00,2011,CST,VOLATILE ORGANIC COMPOUNDS (VOCS),PSS


In [29]:
formalact.head()

,PGM_SYS_ID,SETTLEMENT_ENTERED_DATE,SETTLEMENT_ENTERED_DATE_year,ENF_TYPE_CODE,PENALTY_AMOUNT
2,IL000197800AAA,2009-01-27,2009,CIV,0.0
4,IL000021816AAQ,2015-04-07,2015,CIV,0.0
5,OH0000000374010102,2015-04-07,2015,CIV,0.0
6,IL000143065AFM,2015-04-07,2015,CIV,0.0
7,IL000095020ABS,2015-04-07,2015,CIV,0.0


In [30]:
informalact.head()

,PGM_SYS_ID,ACHIEVED_DATE,ACHIEVED_DATE_year,ENF_TYPE_CODE
84,AR0000000506900118,2007-08-10 00:00:00,2007,NOV
86,TX0000004844900005,2008-02-05 00:00:00,2008,NOV
87,LA0000002200500046,2008-06-26 00:00:00,2008,NOV
89,0900000006071R9800,2008-03-18 00:00:00,2008,NOV
90,0900000006099R9800,2008-03-08 00:00:00,2008,NOV


In [63]:
pd_violations.head()

,PGM_SYS_ID,ACTIVITY_ID,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,COMP_DETERMINATION_UID,ENF_RESPONSE_POLICY_CODE,PROGRAM_CODES,PROGRAM_DESCS,POLLUTANT_CODES,POLLUTANT_DESCS,EARLIEST_FRV_DETERM_DATE,HPV_DAYZERO_DATE,HPV_RESOLVED_DATE
0,CT0000000900700108,3400302038,State,CT,NaN,CT000A0000090070010800028,HPV,CAASIP,State Implementation Plan for National Primary...,300000005 300000323,NITROGEN OXIDES NO2 POLLUTANT X,NaN,05-28-1996,10-15-1996
1,CT0000000900700108,3400302043,State,CT,NaN,CT000A0000090070010800158,FRV,CAATVP,Title V Permits,300000282,NITROGEN OXIDES,05-24-2013,NaN,NaN
2,CT0000000900900110,3400302044,State,CT,NaN,CT000A0000090090011000026,HPV,CAASIP,State Implementation Plan for National Primary...,300000005 300000323,NITROGEN OXIDES NO2 POLLUTANT X,NaN,05-28-1996,10-15-1996
3,CT0000000900900110,3400302045,State,CT,NaN,CT000A0000090090011000037,HPV,CAASIP,State Implementation Plan for National Primary...,10461 300000005,NITROGEN OXIDES NO2 Sulfur dioxide,NaN,08-21-1998,09-30-1999
4,CT0000000900300125,3400302055,State,CT,NaN,CT000A0000090030012500017,HPV,CAASIP,State Implementation Plan for National Primary...,300000242 300000329,FACIL TOTAL HAZARDOUS AIR POLLUTANTS (HAPS),NaN,07-12-2005,07-17-2007


In [59]:
pd_inspection.head()

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES
0,MI00000000000A1932,3600154506,E,INS,Inspection/Evaluation,PCE,PCE On-Site,06-09-2015,"CAAFESOP, CAASIP"
1,CASJV00006029S2076,3600154515,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-10-2015,CAATVP
2,CASJV00006039C2587,3600154514,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-08-2015,CAASIP
3,CASJV00006029S1234,3600154512,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-10-2015,CAATVP
4,CASJV00006099N7478,3600154520,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-11-2015,CAATVP


In [58]:
pd_informalact.head()

,PGM_SYS_ID,ACTIVITY_ID,ENF_IDENTIFIER,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,STATE_EPA_FLAG,ENF_TYPE_CODE,ENF_TYPE_DESC,ACHIEVED_DATE
0,NH0000003300900020,119966,01-100020760,AIF,Administrative - Informal,E,LRE,Letter to Regulated Entity,05/24/2004
1,CT0000000900905468,119525,01-100020694,AIF,Administrative - Informal,E,LRE,Letter to Regulated Entity,06/15/2004
2,CT0000000900909912,157284,01-100021969,AIF,Administrative - Informal,E,NOV,Notice of Violation,09/20/2005
3,MA0000002511900793,115568,01-100020398,AIF,Administrative - Informal,E,LRE,Letter to Regulated Entity,04/02/2004
4,CASCA0000605900006,87573,HQ-100018673,AIF,Administrative - Informal,E,NOD,Notice of Determination,12/10/2001


In [57]:
pd_stacktest.head()

,PGM_SYS_ID,ACTIVITY_ID,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,STATE_EPA_FLAG,ACTUAL_END_DATE,POLLUTANT_CODES,POLLUTANT_DESCS,AIR_STACK_TEST_STATUS_CODE,AIR_STACK_TEST_STATUS_DESC
0,RI0000004400900016,600078981,CST,Stack Test,E,10/03/2007,Volatile organic compounds,NaN,NaN,NaN
1,CT0000000900900001,600035891,CST,Stack Test,E,01/17/2007,Volatile organic compounds,NaN,NaN,NaN
2,CT0000000900900001,600035898,CST,Stack Test,E,01/18/2007,Volatile organic compounds,NaN,NaN,NaN
3,ME0000002301300004,600049983,CST,Stack Test,E,05/30/2007,Volatile organic compounds,NaN,NaN,NaN
4,MA0000002511900605,3400446154,CST,Stack Test,S,09/28/2011,VOLATILE ORGANIC COMPOUNDS (VOCS),NaN,PSS,Pass


In [56]:
pd_formalact.head()

,PGM_SYS_ID,ACTIVITY_ID,ENF_IDENTIFIER,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,STATE_EPA_FLAG,ENF_TYPE_CODE,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,PENALTY_AMOUNT
0,020000003400701000,8761,02-2002-1284,AFR,Administrative - Formal,E,113D1,CAA 113D1 Action For Penalty,12/24/2003,500.0
1,NY0000002620201210,6927,02-1999-1264,AFR,Administrative - Formal,E,113D1,CAA 113D1 Action For Penalty,06/06/2000,0.0
2,IL000197800AAA,32608,05-2000-0612,JDC,Judicial,E,CIV,Civil Judicial Action,01/27/2009,0.0
3,IL000197800AAA,32608,05-2000-0612,JDC,Judicial,E,CIV,Civil Judicial Action,12/13/2005,1429089.0
4,IL000021816AAQ,33331,05-2001-0710,JDC,Judicial,E,CIV,Civil Judicial Action,04/07/2015,0.0


In [64]:
pd_titlev.head()

,PGM_SYS_ID,ACTIVITY_ID,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,STATE_EPA_FLAG,ACTUAL_END_DATE,FACILITY_RPT_DEVIATION_FLAG
0,UT0000004903500210,3600154556,TVA,TV ACC Receipt/Review,S,04/30/2015,Y
1,UT0000004903500210,3600154556,TVA,TV ACC Receipt/Review,S,04/30/2015,Y
2,UT0000004903500210,3600154556,TVA,TV ACC Receipt/Review,S,04/30/2015,Y
3,UT0000004903500210,3600154556,TVA,TV ACC Receipt/Review,S,04/30/2015,Y
4,UT0000004903500210,3600154556,TVA,TV ACC Receipt/Review,S,04/30/2015,Y
